<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/Products_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import

In [146]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time

#Manage of files
import pandas as pd
import csv
import numpy as np

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

#For SQL
!pip install psycopg2
import psycopg2

!pip install sqlalchemy
import sqlalchemy as sql

#Go to folder

Hay que considerar que se trabajara con las bases de datos (cualquier formato) y con las carpetas de control de product. Por ello, es que se inicia en la carpeta que permite seleccionar esto. Recordando:
*  Masters: Se encuentran las bases de datos, organizadas por tipo de archivo.
*  Origin: Primeros experimentos, se encuentran los archivos originales por consulta, divididos por país y categoría del país.
*   Products: Dónde se pretende almacenar las base de características de productos.

In [5]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
Masters  Origin  Products


#Tools

##Centered Len

#Count if

In [6]:
def count_if(array, value):
    count = 0
    for element in array:
        if element == value:
            count += 1
    return count

##Tools

In [32]:
def mode(array):
    elements = [None]*len(array)
    counter = [None]*len(array)

    for element in array:

        #Check the element
        searcher = 0
        while elements[searcher] != element and elements[searcher] != None and searcher < (len(array) - 1):
            searcher += 1
        
        if elements[searcher] == element:
            counter[searcher] += 1
        
        elif elements[searcher] == None:
            elements[searcher] = element
            counter[searcher] = 1

    max = 0
    c = 0
    while counter[c] != None and c<(len(counter)-1):
        if counter[c] >= max:
            max = counter[c]
            max_position = c
        c += 1

    mode = elements[max_position]
    return mode

#Standarize dates

In [7]:
def standarize_dates(date):
    check = False
    try:
        date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S.%f')
        check = True
    except ValueError:
        try:
            date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
            check = True
        except:
            try:
                date = datetime.strptime(str(date), '%Y-%m-%d %H_%M')
                check = True
            except:
                pass
    if check == True:
        date = date.strftime('%Y-%m-%d %H:%M')
    
    return date

#Lab

##Extract Products

###Name Products

In [97]:
entry = 'Masters/mx/parquet/mx-master_db_music.parquet'

#Get de Nanmes
df = pd.read_parquet(entry)
df['time'] = df['time'].apply(standarize_dates)
df = df.sort_values(by="time")
product_names_raw = df['Product Names'].tolist()
#Remember, this gets de None Value
products = list(set(product_names_raw))
None_cases = count_if(product_names_raw, None)
products.remove(None)

##Scrap Dates

In [130]:
dates_raw = df['time'].tolist()
dates_time = set(dates_raw)

just_dates = []
for dt in dates_time:
    try:
        date = datetime.strptime(dt,'%Y-%m-%d %H:%M').date()
    except:
        date = datetime.strptime(dt,'%Y-%m-%d %H_%M').date()

    if not date in just_dates:
        just_dates.append(date)

just_dates.sort()
just_dates

[datetime.date(2020, 7, 31),
 datetime.date(2020, 8, 1),
 datetime.date(2020, 8, 2),
 datetime.date(2020, 8, 3),
 datetime.date(2020, 8, 4),
 datetime.date(2020, 8, 5),
 datetime.date(2020, 8, 6),
 datetime.date(2020, 8, 7),
 datetime.date(2020, 8, 8),
 datetime.date(2020, 8, 9),
 datetime.date(2020, 8, 10),
 datetime.date(2020, 8, 11),
 datetime.date(2020, 8, 12),
 datetime.date(2020, 8, 13)]

#Obtaining All Mode Ranks in table

In [ ]:
products_dict = {products[n]:n for n in range(len(products))}
# products_dict

In [149]:
ranks = df['Rank'].tolist()
day_arrays = [None] * (len(just_dates) - 1)
zero_rank_day_array = [None] * len(products)
rank_day_array = [None] * len(products)

d_counter = 0
first_date = just_dates[0]

for r in range(len(ranks)):
    
    #Day that we are
    try:
        date_check = datetime.strptime(dates_raw[r],'%Y-%m-%d %H:%M').date()
    except:
        date_check = datetime.strptime(dates_raw[r],'%Y-%m-%d %H_%M').date()

    if date_check != first_date:
        day_arrays[d_counter] = rank_day_array
        rank_day_array = [None] * len(products)
        if d_counter < (len(just_dates)-1):
            d_counter += 1
            first_date = just_dates[d_counter]

    if date_check == first_date:
        # if date_check == just_dates[0]:
        #     print(date_check)
        rank = ranks[r]
        #What product are we talking about
        product_name = product_names_raw[r]
        if product_name != None:
            product_position = products_dict[product_name]
            if rank_day_array[product_position] == None:
                array = []
                rank_day_array[product_position] = array
            else:
                array_rank = rank_day_array[product_position]
                array_rank.append(rank)
        
            # print(f'{product_name} Position: {product_position} rank: {rank} -> {rank_day_array[product_position]}')

for ranked_day in day_arrays:
    print(ranked_day)

print('\n')

for ranked_day in day_arrays:
    for r in range(len(ranked_day)):
        product = ranked_day[r]
        if product != None:
            if len(product) > 1:
                moda = mode(ranked_day[r])
                ranked_day[r] = moda
            elif len(product) == 0:
                ranked_day[r] = None
            elif len(product) == 1:
                ranked_day[r] = product[0]

for ranked_day in day_arrays:
    print(ranked_day)


[None, None, None, None, None, None, None, [47.0, 47.0, 47.0], None, None, None, None, [39.0, 39.0, 39.0, 39.0, 39.0], None, None, None, None, None, [27.0, 27.0, 27.0, 41.0, 41.0, 41.0, 41.0, 41.0], [], None, None, [], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0], None, None, None, None, None, None, None, [22.0, 22.0, 22.0, 22.0, 34.0, 34.0, 34.0, 34.0, 34.0], None, None, None, None, None, None, None, [45.0, 45.0, 45.0, 45.0], None, None, None, None, [11.0, 11.0, 11.0, 7.0, 7.0, 7.0, 7.0, 7.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0], None, None, None, None, None, None, None, None, [5.0, 5.0, 5.0, 5.0, 8.0, 8.0, 8.0, 8.0, 8.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [50.0, 50.0, 50.0, 50.0], None, None, None, None, None, [], None, None, [26.0, 26.0, 26.0, 39.0, 39.0, 39.0, 39.0, 39.0], [], None, None, None, None, None, [], None, None, None, None, None, None, None, [19.0, 19.

##Query for product

In [138]:
products[10]

'Re'

In [148]:
database_type = 'mysql'

user = 'user_name'
password = 'password'
host = 'xxx.xxx.xxx.xxx:port'
database = 'database_name'

conn_string = '{}://{}:{}@{}/{}'.format(database_type, user, password, host, database)

sql_conn = sql.create_engine(conn_string)

ValueError: ignored

In [ ]:
query_sql = '''
SELECT time, Product Names, rank
from 
'''

In [144]:
music_db = df
music_db

,time,Rank,Product Names,Image urls,Product links,Stars,Reviews,Authors/Company,Edition/Console,Price_std_or_min,Max_prices
3763,2020-07-31 00:38,14.0,Map Of The Soul: Persona,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Map-Soul-Persona-BTS...,4.8,2464.0,BTS,Audio CD,432.86,None
3773,2020-07-31 00:38,24.0,"Love Yourself: Her (Random cover, incl. 100-pa...",https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Love-Yourself-Her-BT...,4.7,900.0,BTS,Audio CD,688.00,None
3772,2020-07-31 00:38,23.0,Harry Styles (Edición Especial) (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Harry-Styles-Edici%C...,4.7,1388.0,Harry Styles,Vinil,1223.28,None
3771,2020-07-31 00:38,22.0,Blurryface (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Blurryface-Vinyl-Twe...,4.7,2255.0,Twenty One Pilots,Vinil,561.00,None
3769,2020-07-31 00:38,20.0,Up All Night,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Up-All-Night-One-Dir...,4.7,1108.0,One Direction,Audio CD,127.00,None
...,...,...,...,...,...,...,...,...,...,...,...
19,2020-08-13 20:46,20.0,Is This It (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/This-Vinyl-Strokes/d...,4.3,575.0,The Strokes,Vinil,392.25,None
20,2020-08-13 20:46,21.0,Caifanes,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Caifanes/dp/B00WKNGH...,4.4,132.0,Caifanes,Audio CD,99.00,None
21,2020-08-13 20:46,22.0,Black Pumas (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Black-Pumas-Vinyl/dp...,NaN,NaN,Black Pumas,Vinil,1162.03,None
11,2020-08-13 20:46,12.0,Dynasty (Vinyl) [Importado],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Dynasty-Vinyl-Import...,4.6,239.0,Kiss,Vinil,NaN,None
